<a href="https://colab.research.google.com/github/TrzeciakPiotr2300/Uczenie_Maszynowe_2025/blob/main/HW_lab12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###***Step one, set up the environment*** ###

In [6]:
# Install stable versions to avoid numpy-related bugs
!pip install --upgrade --quiet gensim==4.3.0 scikit-learn==1.4.1.post1 numpy==1.24.4 matplotlib==3.7.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.8 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.24 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
db-dtypes 1.4.3 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.

In [1]:
# Import libraries
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm

ImportError: cannot import name 'triu' from 'scipy.linalg' (/usr/local/lib/python3.11/dist-packages/scipy/linalg/__init__.py)

In [8]:
# Load pre-trained GloVe embeddings (100-dimensional)
embedding_model = api.load("glove-wiki-gigaword-100")

# Helper function to fetch vector
def get_vector(word):
    return embedding_model[word] if word in embedding_model else None

# Helper function to fetch multiple vectors
def get_vectors(words):
    return np.array([get_vector(word) for word in words if get_vector(word) is not None])


NameError: name 'api' is not defined

##**Define semantic words classes**

In [ ]:
# Define 10 semantic classes, each with 5 word pairs
semantic_classes = {
    "gender": [
        ("king", "queen"),
        ("man", "woman"),
        ("boy", "girl"),
        ("husband", "wife"),
        ("actor", "actress")
    ],
    "past_tense": [
        ("run", "ran"),
        ("go", "went"),
        ("write", "wrote"),
        ("eat", "ate"),
        ("speak", "spoke")
    ],
    "plural": [
        ("dog", "dogs"),
        ("car", "cars"),
        ("child", "children"),
        ("mouse", "mice"),
        ("foot", "feet")
    ],
    "comparative": [
        ("fast", "faster"),
        ("small", "smaller"),
        ("happy", "happier"),
        ("strong", "stronger"),
        ("bright", "brighter")
    ],
    "superlative": [
        ("fast", "fastest"),
        ("small", "smallest"),
        ("happy", "happiest"),
        ("strong", "strongest"),
        ("bright", "brightest")
    ],
    "country_capital": [
        ("france", "paris"),
        ("italy", "rome"),
        ("germany", "berlin"),
        ("japan", "tokyo"),
        ("egypt", "cairo")
    ],
    "currency": [
        ("usa", "dollar"),
        ("japan", "yen"),
        ("uk", "pound"),
        ("france", "euro"),
        ("china", "yuan")
    ],
    "tool_user": [
        ("pen", "writer"),
        ("hammer", "carpenter"),
        ("brush", "painter"),
        ("scalpel", "surgeon"),
        ("camera", "photographer")
    ],
    "job_object": [
        ("chef", "pan"),
        ("driver", "car"),
        ("farmer", "tractor"),
        ("pianist", "piano"),
        ("teacher", "book")
    ],
    "animal_sound": [
        ("dog", "bark"),
        ("cat", "meow"),
        ("cow", "moo"),
        ("duck", "quack"),
        ("lion", "roar")
    ]
}


In [ ]:
### Now we gather all unique words and their vectors:
# Flatten to get all unique words
all_words = set()
for pairs in semantic_classes.values():
    for w1, w2 in pairs:
        all_words.add(w1)
        all_words.add(w2)

# Get vectors only for words present in embedding model
word_vectors = {word: get_vector(word) for word in all_words if get_vector(word) is not None}

# Final word list (with embeddings)
words_with_vectors = list(word_vectors.keys())
print(f"Total unique words with embeddings: {len(words_with_vectors)}")


##**Now we will do PCA analysis to visualise wodr embeddings in a 2D space**##

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
def plot_words_2d(word_list, vectors, title):
    pca = PCA(n_components=2)
    X = np.array([vectors[word] for word in word_list])
    X_pca = pca.fit_transform(X)

    plt.figure(figsize=(16, 12))
    #plt.xlim(-4.1, 7.0)
    #plt.ylim(-3.5, 3.5)
    for i, word in enumerate(word_list):
        plt.scatter(X_pca[i, 0], X_pca[i, 1])
        plt.text(X_pca[i, 0] + 0.02, X_pca[i, 1] + 0.02, word, fontsize=9)
    plt.title(title)
    plt.grid(True)
    plt.show()


In [ ]:
#Global PCA
plot_words_2d(words_with_vectors, word_vectors, "Global PCA: All Words")

In [ ]:
#Global PCA, but one class at a time &&& in a global space:
for class_name, pairs in semantic_classes.items():
    class_words = [w for pair in pairs for w in pair if w in word_vectors]
    plot_words_2d(class_words, word_vectors, f"Global PCA - Class: {class_name}")


In [ ]:
# Local PCA — each class separatelly, so PCA for 10 words in each class.
def plot_local_pca(class_name, pairs):
    class_words = [w for pair in pairs for w in pair if w in word_vectors]
    class_vectors = {word: word_vectors[word] for word in class_words}
    plot_words_2d(class_words, class_vectors, f"Local PCA - Class: {class_name}")

for class_name, pairs in semantic_classes.items():
    plot_local_pca(class_name, pairs)


##**And now we will do some embeddings, meaning calculate expressions such as: embedding("king") - embedding("man") + embedding("woman") ~= "queen"**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
def find_closest_words(vec, vectors_dict, topn=5):
    all_words = list(vectors_dict.keys())
    all_vecs = np.array([vectors_dict[w] for w in all_words])

    sims = cosine_similarity([vec], all_vecs)[0]
    sorted_indices = np.argsort(sims)[::-1]

    results = []
    for idx in sorted_indices[:topn]:
        results.append((all_words[idx], sims[idx]))
    return results


In [ ]:
#Calculations for every class:
for class_name, pairs in semantic_classes.items():
    base_pair = pairs[0]  # (A, B)
    alt_pair = pairs[1]   # (C, D), weźmiemy C

    A, B = base_pair
    C, _ = alt_pair

    if A in word_vectors and B in word_vectors and C in word_vectors:
        result_vec = word_vectors[B] - word_vectors[A] + word_vectors[C]
        top_matches = find_closest_words(result_vec, word_vectors)

        print(f"\n=== Class: {class_name} ===")
        print(f"Operation: {B} - {A} + {C}")
        for word, score in top_matches:
            print(f"{word:<15} similarity: {score:.4f}")
    else:
        print(f"\nSkipping class '{class_name}' due to missing word(s)")


##**Conclusion for second part of discussion (for PCA plots, we will make another 10 plots)**

In [ ]:
# Perform global PCA for all words with vectors
global_pca = PCA(n_components=2)
words = list(word_vectors.keys())
vectors = np.array([word_vectors[word] for word in words])
reduced_embeddings = global_pca.fit_transform(vectors)

# Create a dictionary: word -> global PCA coordinates
global_pca_dict = {word: reduced_embeddings[i] for i, word in enumerate(words)}

# Plot overlay charts for each class (global vs local PCA)
for class_name, pairs in semantic_classes.items():
    words_in_class = [word for pair in pairs for word in pair if word in word_vectors]

    # Skip class if any word is missing (to avoid KeyError)
    if not all(word in global_pca_dict for word in words_in_class):
        print(f"Skipping {class_name}: missing words in global PCA")
        continue

    # Get global PCA coordinates for words in this class
    global_coords = np.array([global_pca_dict[word] for word in words_in_class])

    # Compute local PCA for just the words in this class
    local_vectors = np.array([word_vectors[word] for word in words_in_class])
    pca_local = PCA(n_components=2)
    local_coords = pca_local.fit_transform(local_vectors)

    # Plotting the overlay chart
    plt.figure(figsize=(10, 7))
    for i, word in enumerate(words_in_class):
        # Global: blue circles
        plt.scatter(global_coords[i, 0], global_coords[i, 1], color='blue', marker='o', label='Global PCA' if i == 0 else "")
        plt.text(global_coords[i, 0]+0.01, global_coords[i, 1]+0.01, word, fontsize=9, color='blue')

        # Local: red crosses
        plt.scatter(local_coords[i, 0], local_coords[i, 1], color='red', marker='x', label='Local PCA' if i == 0 else "")
        plt.text(local_coords[i, 0]+0.01, local_coords[i, 1]+0.01, word, fontsize=9, color='red')

    plt.title(f"PCA Global vs Local — {class_name}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


NameError: name 'reduced_embeddings' is not defined